# An offline demo for the "Learning-based Video Motion Magnification" (ECCV 2018)

# Official repo: https://github.com/12dmodel/deep_motion_mag
# My repo: https://github.com/ZhengPeng7/motion_magnification_learning-based

## Make sure to set your python Kernel for notebook. 

## Preparations:


### Install python packages

In [1]:
!pip install --quiet -r requirements.txt
!pip install --quiet gdown mediapy


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Download and load the well-trained weights:

In [3]:
!wget https://github.com/ZhengPeng7/motion_magnification_learning-based/releases/download/v1.0/magnet_epoch12_loss7.28e-02.pth

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [2]:
from magnet import MagNet
from callbacks import gen_state_dict
from config import Config


# config
config = Config()
# Load weights
weights_path = 'magnet_epoch12_loss7.28e-02.pth'
ep = int(weights_path.split('epoch')[-1].split('_')[0])
state_dict = gen_state_dict(weights_path)

model_test = MagNet().cuda()
model_test.load_state_dict(state_dict)
model_test.eval()
print("Loading weights:", weights_path)

Please load train_mf.txt if you want to do training.
Loading weights: magnet_epoch12_loss7.28e-02.pth


# Preprocess

Make the video to frameAs/frameBs/frameCs.

Let's take the guitar.mp4 as an example.

In [3]:
# Download some example video from my google-drive or upload your own video.
# !gdown 1hNZ02vnSO04FYS9jkx2OjProYHIvwdkB  # guitar.avi
!gdown 1XGC2y4Lshd9aBiBxwkTuT_IA79n-WNST  
# baby.avi
# !gdown 1QGOWuR0swF7_eHharTztlkEDz0hlfmU4  # zhiyin.mp4

Traceback (most recent call last):
  File "C:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\alexi\AppData\Local\Programs\Python\Python310\Scripts\gdown.exe\__main__.py", line 7, in <module>
  File "C:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\gdown\cli.py", line 156, in main
    filename = download(
  File "C:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\gdown\download.py", line 259, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'


# Set VIDEO_NAME here, e.g., guitar, baby

In [7]:
# Turn the video into frames and make them into frame_ACB format.
file_to_be_maged = "bridge_2.mp4"#'baby.avi'   # 'guitar.avi'
video_name = file_to_be_maged.split('.')[0]
video_format = '.' + file_to_be_maged.split('.')[-1]


sh_file = 'VIDEO_NAME={}\nVIDEO_FORMAT={}'.format(video_name, video_format) + """


mkdir ${VIDEO_NAME}
ffmpeg -i ${VIDEO_NAME}${VIDEO_FORMAT} -f image2 ${VIDEO_NAME}/%06d.png
python make_frameACB.py ${VIDEO_NAME}
mkdir test_dir
mv ${VIDEO_NAME} test_dir
"""
with open('test_preproc.sh', 'w') as file:
  file.write(sh_file)
!bash test_preproc.sh

test_preproc.sh: line 3: $'\r': command not found
test_preproc.sh: line 4: $'\r': command not found
test_preproc.sh: line 6: ffmpeg: command not found
test_preproc.sh: line 7: python: command not found
mkdir: cannot create directory ‘test_dir\r’: File exists
mv: cannot stat 'bridge_2'$'\r': No such file or directory


## Test


In [9]:
import os
import sys
import cv2
import torch
import numpy as np
from data import get_gen_ABC, unit_postprocessing, numpy2cuda, resize2d

# testsets = '+'.join(['baby', 'guitar', 'gun', 'drone', 'cattoy', 'water'][:])
video_name="guitar"
for testset in [video_name]:
    dir_results = 'res_' + testset
    if not os.path.exists(dir_results):
        os.makedirs(dir_results)

    config.data_dir = 'test_dir'
    data_loader = get_gen_ABC(config, mode='test_on_'+testset)
    print('Number of test image couples:', data_loader.data_len)
    vid_size = cv2.imread(data_loader.paths[0]).shape[:2][::-1]

    # Test
    for amp in [10, 25, 50]:
        frames = []
        data_loader = get_gen_ABC(config, mode='test_on_'+testset)
        for idx_load in range(0, data_loader.data_len, data_loader.batch_size):
            if (idx_load+1) % 100 == 0:
                print('{}'.format(idx_load+1), end=', ')
            batch_A, batch_B = data_loader.gen_test()
            amp_factor = numpy2cuda(amp)
            for _ in range(len(batch_A.shape) - len(amp_factor.shape)):
                amp_factor = amp_factor.unsqueeze(-1)
            with torch.no_grad():
                y_hats = model_test(batch_A, batch_B, 0, 0, amp_factor, mode='evaluate')
            for y_hat in y_hats:
                y_hat = unit_postprocessing(y_hat, vid_size=vid_size)
                frames.append(y_hat)
                if len(frames) >= data_loader.data_len:
                    break
            if len(frames) >= data_loader.data_len:
                break
        data_loader = get_gen_ABC(config, mode='test_on_'+testset)
        frames = [unit_postprocessing(data_loader.gen_test()[0], vid_size=vid_size)] + frames

        # Make videos of framesMag
        video_dir = os.path.join(dir_results, testset)
        if not os.path.exists(video_dir):
            os.makedirs(video_dir)
        FPS = 30
        video_save_path = os.path.join(video_dir, '{}_amp{}{}'.format(testset, amp, video_format))
        out = cv2.VideoWriter(
            video_save_path,
            cv2.VideoWriter_fourcc(*'DIVX'),
            FPS, frames[0].shape[-2::-1]
        )
        for frame in frames:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            cv2.putText(frame, 'amp_factor={}'.format(amp), (7, 37),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
            out.write(frame)
        out.release()
        print('{} has been done.'.format(video_save_path))

Number of test image couples: 113
100, res_bridge_2\bridge_2\bridge_2_amp10.mp4 has been done.
100, res_bridge_2\bridge_2\bridge_2_amp25.mp4 has been done.
100, res_bridge_2\bridge_2\bridge_2_amp50.mp4 has been done.


In [ ]:
# Play the amplified video here
from glob import glob
import mediapy


video_save_paths = [file_to_be_maged] + sorted(glob(os.path.join(dir_results, testset, '*')), key=lambda x: int(x.split('amp')[-1].split('.')[0]))

video_dict = {}
for video_save_path in video_save_paths[:]:
  video_dict[video_save_path.split('/')[-1]] = mediapy.read_video(video_save_path)
mediapy.show_videos(video_dict, fps=FPS, width=250, codec='gif')

# II/ Frequency from Video


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\asyncio\selector_events.py", line 115, in _read_from_self

: 

Please load train_mf.txt if you want to do training.
Save_dir: weights_date0510


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '../../../datasets/mm\\train\\frameA'